In [146]:
## Exercise 1.)

# loading the csv file
import boto3
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV


s3= boto3.resource('s3')
bucket_name= 'macklins-bucket'
bucket= s3.Bucket(bucket_name)

file_key= 'College.csv'

bucket_object= bucket.Object(file_key)
file_object= bucket_object.get()
file_content_stream= file_object.get('Body')

## Reading the data file
college= pd.read_csv(file_content_stream)
college.head()

,Private,Apps,Accept,Enroll,Top10perc,Top25perc,F.Undergrad,P.Undergrad,Outstate,Room.Board,Books,Personal,PhD,Terminal,S.F.Ratio,perc.alumni,Expend,Grad.Rate
Abilene Christian University,Yes,1660,1232,721,23,52,2885,537,7440,3300,450,2200,70,78,18.1,12,7041,60
Adelphi University,Yes,2186,1924,512,16,29,2683,1227,12280,6450,750,1500,29,30,12.2,16,10527,56
Adrian College,Yes,1428,1097,336,22,50,1036,99,11250,3750,400,1165,53,66,12.9,30,8735,54
Agnes Scott College,Yes,417,349,137,60,89,510,63,12960,5450,450,875,92,97,7.7,37,19016,59
Alaska Pacific University,Yes,193,146,55,16,44,249,869,7560,4120,800,1500,76,72,11.9,2,10922,15


In [147]:
# b.)
# changing Private variable to a zero to one scale
college['Private'] = np.where(college['Private'] == 'Yes', 0, 1)
college.head()

,Private,Apps,Accept,Enroll,Top10perc,Top25perc,F.Undergrad,P.Undergrad,Outstate,Room.Board,Books,Personal,PhD,Terminal,S.F.Ratio,perc.alumni,Expend,Grad.Rate
Abilene Christian University,0,1660,1232,721,23,52,2885,537,7440,3300,450,2200,70,78,18.1,12,7041,60
Adelphi University,0,2186,1924,512,16,29,2683,1227,12280,6450,750,1500,29,30,12.2,16,10527,56
Adrian College,0,1428,1097,336,22,50,1036,99,11250,3750,400,1165,53,66,12.9,30,8735,54
Agnes Scott College,0,417,349,137,60,89,510,63,12960,5450,450,875,92,97,7.7,37,19016,59
Alaska Pacific University,0,193,146,55,16,44,249,869,7560,4120,800,1500,76,72,11.9,2,10922,15


In [148]:
# c.)
# Creating the target and predictor variables

# creating the two data-frames
from sklearn.model_selection import train_test_split

# Definning the inputs and target variable
X= college[['Private', 'F.Undergrad', 'P.Undergrad', 'Outstate', 'Room.Board', 'Books', 'Personal', 'S.F.Ratio', 'Grad.Rate']]
Y = college['Apps']
# Splitting the data
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2)

In [149]:
# d.)
from sklearn.preprocessing import MinMaxScaler

# transforming the data to a 0-1 scale
scaler= MinMaxScaler()
X_train= scaler.fit_transform(X_train)
X_test= scaler.fit_transform(X_test)

In [150]:
# e.)
# building the linear regression model

# Importing LinearRegression
from sklearn.linear_model import LinearRegression

# Fitting a simple linear regression model on train and test sets
lm_md = LinearRegression().fit(X_train, Y_train)

# Making the prediction
lm_pred = lm_md.predict(X_test)

# computing the mse of the linear regression model
mse1 = np.mean(np.power(Y_test - lm_pred, 2))
print('The mse of the linear model is:', mse1)

The mse of the linear model is: 2860009.6520753894


In [151]:
# f.)

# Building the ridge regression model

# Importing Ridge and RidgeCV
from sklearn.linear_model import Ridge, RidgeCV
# Running cross-validation with different values for lambda (using 5-folds)

ridge_cv = RidgeCV(np.linspace(0.001,100, num=100), cv =5).fit(X_train, Y_train)
# extracting the best lamba
cv_lambda= ridge_cv.alpha_
# building the ridge model
ridge_md= Ridge(alpha = cv_lambda).fit(X_train, Y_train)
# predicting on test
ridge_pred = ridge_md.predict(X_test)

In [152]:
# computing the mse of the model
mse_ridge = np.mean(np.power(Y_test - ridge_pred, 2))
print('The mse of the ridge model is:', mse_ridge)

The mse of the ridge model is: 2853116.057110594


In [153]:
# g.)
# building the lasso regression model
from sklearn.linear_model import Lasso, LassoCV
# Running cross-validation with different values for lambda (using 5-folds)
lasso_cv = LassoCV(normalize = True, cv = 5).fit(X,Y)
# Extracting the optimal alpha
CV_alpha = lasso_cv.alpha_
# Re-fitting the model with optimal alpha
lasso_md = Lasso(alpha = CV_alpha, normalize = True).fit(X, Y)
lasso_md.coef_

# making the prediction
lasso_pred = lasso_md.predict(X_test)

# Displaying results
print('The estimated lamba for the lasso model is', cv_lambda)

The estimated lamba for the lasso model is 1.011090909090909


/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/sklearn/linear_model/_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Lasso())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
  warnings.warn(
/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/sklearn/linear_model/_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing

In [154]:
# computing the mse of the ridge regression model
mse_lasso = np.mean(np.power(Y_test - lasso_pred, 2))
print('The mse of the lasso model is:', mse_lasso)

The mse of the lasso model is: 55704462.664718434


In [155]:
# h.) 
# The mse of the ridge model is the lowest, and is most appropriate to be selected.

In [156]:
## Exercise 2.)
# a.)
import pandas as pd
import numpy as np

from sklearn.metrics import recall_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import StratifiedKFold

# Reading the churn-bigml-80.csv
s3= boto3.resource('s3')
bucket_name= 'macklins-bucket'
bucket= s3.Bucket(bucket_name)

file_key= 'churn-bigml-80.csv'

bucket_object= bucket.Object(file_key)
file_object= bucket_object.get()
file_content_stream= file_object.get('Body')

## Reading the data file
telecom_train= pd.read_csv(file_content_stream)
telecom_train.head()

,State,Account_length,Area_code,International_plan,Voice_mail_plan,Number_vmail_messages,Total_day_minutes,Total_day_calls,Total_day_charge,Total_eve_minutes,Total_eve_calls,Total_eve_charge,Total_night_minutes,Total_night_calls,Total_night_charge,Total_intl_minutes,Total_intl_calls,Total_intl_charge,Customer_service_calls,Churn
0,KS,128,415,No,Yes,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,OH,107,415,No,Yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,NJ,137,415,No,No,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,OH,84,408,Yes,No,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,OK,75,415,Yes,No,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False


In [157]:
# Reading the churn-bigml-20.csv
s3= boto3.resource('s3')
bucket_name= 'macklins-bucket'
bucket= s3.Bucket(bucket_name)

file_key= 'churn-bigml-20.csv'

bucket_object= bucket.Object(file_key)
file_object= bucket_object.get()
file_content_stream= file_object.get('Body')

## Reading the data file
telecom_test= pd.read_csv(file_content_stream)
telecom_test.head()

,State,Account_length,Area_code,International_plan,Voice_mail_plan,Number_vmail_messages,Total_day_minutes,Total_day_calls,Total_day_charge,Total_eve_minutes,Total_eve_calls,Total_eve_charge,Total_night_minutes,Total_night_calls,Total_night_charge,Total_intl_minutes,Total_intl_calls,Total_intl_charge,Customer_service_calls,Churn
0,LA,117,408,No,No,0,184.5,97,31.37,351.6,80,29.89,215.8,90,9.71,8.7,4,2.35,1,False
1,IN,65,415,No,No,0,129.1,137,21.95,228.5,83,19.42,208.8,111,9.40,12.7,6,3.43,4,True
2,NY,161,415,No,No,0,332.9,67,56.59,317.8,97,27.01,160.6,128,7.23,5.4,9,1.46,4,True
3,SC,111,415,No,No,0,110.4,103,18.77,137.3,102,11.67,189.6,105,8.53,7.7,6,2.08,2,False
4,HI,49,510,No,No,0,119.3,117,20.28,215.1,109,18.28,178.7,90,8.04,11.1,1,3.00,1,False


In [158]:
# b.) 
# changing the churn variable from 0-1
telecom_test['Churn'] = np.where(telecom_test['Churn'] == False, 0, 1)
telecom_test.head()

,State,Account_length,Area_code,International_plan,Voice_mail_plan,Number_vmail_messages,Total_day_minutes,Total_day_calls,Total_day_charge,Total_eve_minutes,Total_eve_calls,Total_eve_charge,Total_night_minutes,Total_night_calls,Total_night_charge,Total_intl_minutes,Total_intl_calls,Total_intl_charge,Customer_service_calls,Churn
0,LA,117,408,No,No,0,184.5,97,31.37,351.6,80,29.89,215.8,90,9.71,8.7,4,2.35,1,0
1,IN,65,415,No,No,0,129.1,137,21.95,228.5,83,19.42,208.8,111,9.40,12.7,6,3.43,4,1
2,NY,161,415,No,No,0,332.9,67,56.59,317.8,97,27.01,160.6,128,7.23,5.4,9,1.46,4,1
3,SC,111,415,No,No,0,110.4,103,18.77,137.3,102,11.67,189.6,105,8.53,7.7,6,2.08,2,0
4,HI,49,510,No,No,0,119.3,117,20.28,215.1,109,18.28,178.7,90,8.04,11.1,1,3.00,1,0


In [159]:
telecom_train['Churn'] = np.where(telecom_train['Churn'] == False, 0, 1)
telecom_train.head()

,State,Account_length,Area_code,International_plan,Voice_mail_plan,Number_vmail_messages,Total_day_minutes,Total_day_calls,Total_day_charge,Total_eve_minutes,Total_eve_calls,Total_eve_charge,Total_night_minutes,Total_night_calls,Total_night_charge,Total_intl_minutes,Total_intl_calls,Total_intl_charge,Customer_service_calls,Churn
0,KS,128,415,No,Yes,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,0
1,OH,107,415,No,Yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,0
2,NJ,137,415,No,No,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,0
3,OH,84,408,Yes,No,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,0
4,OK,75,415,Yes,No,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,0


In [160]:
telecom_train['International_plan'] = np.where(telecom_train.Churn ==  False, 0, 1)
telecom_train.head()

,State,Account_length,Area_code,International_plan,Voice_mail_plan,Number_vmail_messages,Total_day_minutes,Total_day_calls,Total_day_charge,Total_eve_minutes,Total_eve_calls,Total_eve_charge,Total_night_minutes,Total_night_calls,Total_night_charge,Total_intl_minutes,Total_intl_calls,Total_intl_charge,Customer_service_calls,Churn
0,KS,128,415,0,Yes,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,0
1,OH,107,415,0,Yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,0
2,NJ,137,415,0,No,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,0
3,OH,84,408,0,No,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,0
4,OK,75,415,0,No,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,0


In [161]:
# changing Voice_mail_plan 
telecom_test['Voice_mail_plan'] = np.where(telecom_test.Churn ==  False, 0, 1)
telecom_test.head()

,State,Account_length,Area_code,International_plan,Voice_mail_plan,Number_vmail_messages,Total_day_minutes,Total_day_calls,Total_day_charge,Total_eve_minutes,Total_eve_calls,Total_eve_charge,Total_night_minutes,Total_night_calls,Total_night_charge,Total_intl_minutes,Total_intl_calls,Total_intl_charge,Customer_service_calls,Churn
0,LA,117,408,No,0,0,184.5,97,31.37,351.6,80,29.89,215.8,90,9.71,8.7,4,2.35,1,0
1,IN,65,415,No,1,0,129.1,137,21.95,228.5,83,19.42,208.8,111,9.40,12.7,6,3.43,4,1
2,NY,161,415,No,1,0,332.9,67,56.59,317.8,97,27.01,160.6,128,7.23,5.4,9,1.46,4,1
3,SC,111,415,No,0,0,110.4,103,18.77,137.3,102,11.67,189.6,105,8.53,7.7,6,2.08,2,0
4,HI,49,510,No,0,0,119.3,117,20.28,215.1,109,18.28,178.7,90,8.04,11.1,1,3.00,1,0


In [162]:
telecom_train['Voice_mail_plan'] = np.where(telecom_train.Churn ==  False, 0, 1)
telecom_train.head()

,State,Account_length,Area_code,International_plan,Voice_mail_plan,Number_vmail_messages,Total_day_minutes,Total_day_calls,Total_day_charge,Total_eve_minutes,Total_eve_calls,Total_eve_charge,Total_night_minutes,Total_night_calls,Total_night_charge,Total_intl_minutes,Total_intl_calls,Total_intl_charge,Customer_service_calls,Churn
0,KS,128,415,0,0,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,0
1,OH,107,415,0,0,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,0
2,NJ,137,415,0,0,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,0
3,OH,84,408,0,0,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,0
4,OK,75,415,0,0,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,0


In [163]:
# creating the total_charge variable
telecom_test['total_charge'] = (telecom_test['Total_day_charge']+telecom_test['Total_eve_charge'] + telecom_test['Total_night_charge']+ telecom_test['Total_intl_charge'])
telecom_test.head()

,State,Account_length,Area_code,International_plan,Voice_mail_plan,Number_vmail_messages,Total_day_minutes,Total_day_calls,Total_day_charge,Total_eve_minutes,Total_eve_calls,Total_eve_charge,Total_night_minutes,Total_night_calls,Total_night_charge,Total_intl_minutes,Total_intl_calls,Total_intl_charge,Customer_service_calls,Churn,total_charge
0,LA,117,408,No,0,0,184.5,97,31.37,351.6,80,29.89,215.8,90,9.71,8.7,4,2.35,1,0,73.32
1,IN,65,415,No,1,0,129.1,137,21.95,228.5,83,19.42,208.8,111,9.40,12.7,6,3.43,4,1,54.20
2,NY,161,415,No,1,0,332.9,67,56.59,317.8,97,27.01,160.6,128,7.23,5.4,9,1.46,4,1,92.29
3,SC,111,415,No,0,0,110.4,103,18.77,137.3,102,11.67,189.6,105,8.53,7.7,6,2.08,2,0,41.05
4,HI,49,510,No,0,0,119.3,117,20.28,215.1,109,18.28,178.7,90,8.04,11.1,1,3.00,1,0,49.60


In [164]:
telecom_train['total_charge'] = (telecom_train['Total_day_charge']+telecom_train['Total_eve_charge'] + telecom_train['Total_night_charge']+ telecom_train['Total_intl_charge'])
telecom_train.head()

,State,Account_length,Area_code,International_plan,Voice_mail_plan,Number_vmail_messages,Total_day_minutes,Total_day_calls,Total_day_charge,Total_eve_minutes,Total_eve_calls,Total_eve_charge,Total_night_minutes,Total_night_calls,Total_night_charge,Total_intl_minutes,Total_intl_calls,Total_intl_charge,Customer_service_calls,Churn,total_charge
0,KS,128,415,0,0,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,0,75.56
1,OH,107,415,0,0,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,0,59.24
2,NJ,137,415,0,0,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,0,62.29
3,OH,84,408,0,0,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,0,66.80
4,OK,75,415,0,0,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,0,52.09


In [165]:
# c.)
# dropping the unneeded variables
telecom_train= telecom_train[['Account_length', 'International_plan', 'Voice_mail_plan', 'total_charge', 'Customer_service_calls','Churn']]
telecom_test= telecom_test[['Account_length', 'International_plan', 'Voice_mail_plan', 'total_charge', 'Customer_service_calls','Churn']]
telecom_train.head()

,Account_length,International_plan,Voice_mail_plan,total_charge,Customer_service_calls,Churn
0,128,0,0,75.56,1,0
1,107,0,0,59.24,1,0
2,137,0,0,62.29,0,0
3,84,0,0,66.80,2,0
4,75,0,0,52.09,3,0


In [166]:
telecom_test.head()

,Account_length,International_plan,Voice_mail_plan,total_charge,Customer_service_calls,Churn
0,117,No,0,73.32,1,0
1,65,No,1,54.20,4,1
2,161,No,1,92.29,4,1
3,111,No,0,41.05,2,0
4,49,No,0,49.60,1,0


In [167]:
# d.)
coef = list()

    # defining the input and target variables
X = telecom_train[['Account_length', 'International_plan', 'Voice_mail_plan', 'total_charge', 'Customer_service_calls']]
Y = telecom_train['Churn']

# 1.)
for i in range(0, 1000): 

    # splitting the data into train and test
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify= Y)
    
    
# 2.)
     
## Transform the input data
    scaler= MinMaxScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.fit_transform(X_test)

# 3.)
# estimating lambda for lasso 
    lasso_cv = LassoCV(cv= 5, normalize= True).fit(X_train, Y_train)


# building the lasso model
    lasso_md= Lasso(alpha= lasso_cv.alpha_).fit(X_train, Y_train)
# appending the coef
    coef.append(lasso_md.coef_)

# making coef a data frame
coef_data= pd.DataFrame(coef)
coef_data.head()

/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/sklearn/linear_model/_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Lasso())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
  warnings.warn(
/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/sklearn/linear_model/_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing

,0,1,2,3,4
0,0.0,0.999939,6.218224e-16,0.0,0.0
1,0.0,0.999939,6.552171e-16,0.0,0.0
2,0.0,0.999939,1.847578e-16,0.0,0.0
3,0.0,0.999939,4.009721e-16,0.0,0.0
4,0.0,0.999939,1.024248e-15,0.0,0.0


In [168]:
# Removing all the columns with variables having more than 200 0's
zero= (coef_data==0).sum()
zero

0    1000
1       0
2     205
3    1000
4    1000
dtype: int64

In [169]:
# From the above data, I can see that I need to drop 3 variables for having over 200 zero variables. Account_lengths, 
# total_charge, Customer_service_calls.

In [170]:
# Removing the variables with more than 200 observations
X= X.drop(columns= ['Account_length'], axis= 1)
X= X.drop(columns= ['total_charge'], axis= 1)
X= X.drop(columns= ['Customer_service_calls'], axis= 1)

In [171]:
X.head()

,International_plan,Voice_mail_plan
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0


In [174]:
# e.)

from sklearn.model_selection import KFold
# creating the for loop for 100 iteractions
for i in range(0,100):
    
    # storing the results
    md_1= list()
    md_2= list()
    md_3= list()
    md_4= list()
    
    # defining the k-folds
    KF= StratifiedKFold(n_splits = 5, shuffle = True)
    
    # making the loop for the k-folds
    for train_ix, test_ix in KF.split(X, Y):
        ## Splitting data into train and validation
        X_train, X_test = X.loc[train_ix], X.loc[test_ix]
        Y_train, Y_test = Y.loc[train_ix], Y.loc[test_ix]
        
        ## scaling the data 
        scaler= MinMaxScaler()
        x_train= scaler.fit_transform(X_train)
        x_test= scaler.fit_transform(X_test)
        
        ##############
        ### MODELS###
        #############
        
        # model 1
        model_1= LinearRegression(solver= 'liblinear', penalty= '11').fit(X_train, Y_train)
        # model 1
        model_2= LinearRegression(solver= 'liblinear', penalty= '12').fit(X_train, Y_train)
        # model 1
        model_3= LinearRegression(solver= 'saga', penalty= '11').fit(X_train, Y_train)
        # model 1
        model_4= LinearRegression(solver= 'saga', penalty= '12').fit(X_train, Y_train)
        ####
        
        ##Making predictions##
        
        # md1
        md1= model_1.predict_proba(X_test)[:,1]
        # md2
        md2= model_2.predict_proba(X_test)[:,1]
        # md3
        md3= model_3.predict_proba(X_test)[:,1]
        # md4
        md4= model_4.predict_proba(X_test)[:,1]
        
        
        ## Computing the MSE of the k-fold
        model_1.append(np.mean((md1 - Y_val)**2))
        model_2.append(np.mean((md1 - Y_val)**2))
        model_3.append(np.mean((md1 - Y_val)**2))
        model_4.append(np.mean((md1 - Y_val)**2))
        
        
## Computing the average MSE
mse1 = np.mean(model_1)
mse2 = np.mean(model_2)
mse3 = np.mean(model_3)
mse4 = np.mean(model_4)

TypeError: __init__() got an unexpected keyword argument 'solver'